In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

c:\Users\Ashwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv('path_to_your_dataset.csv')

# List the columns to be used for generating embeddings, including 'Abstract'
text_columns = ['Abstract', 'Title', 'Keywords', 'Summary']

# Example: Clean your data if necessary
for col in text_columns:
    df[col] = df[col].str.lower().str.replace('[^\w\s]', '')

# Concatenate text columns into a single column
df['CombinedText'] = df[text_columns].fillna('').agg(' '.join, axis=1)


In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the combined text
embeddings = model.encode(df['CombinedText'].tolist(), show_progress_bar=True)

# Add embeddings to the dataframe
df['Embedding'] = list(embeddings)


In [ ]:
import faiss
import numpy as np

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])

# Add embeddings to the index
index.add(embeddings_np)


In [ ]:
def search_entries(query, k=5):
    # Generate embedding for the query
    query_embedding = model.encode([query])

    # Search the index for the most similar embeddings
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)

    # Retrieve the most similar entries
    results = df.iloc[indices[0]].copy()
    results['Distance'] = distances[0]
    return results

# Example search
query = "Your search query here"
search_results = search_entries(query)
print(search_results)
